Setting up the DB

In [ ]:
# -- Switch to postgres user

# sudo -u postgres psql

# -- Create database

# CREATE DATABASE bank_reviews;

# -- Create database user and grant all previlages

# CREATE USER admin WITH PASSWORD 'admin123';

# GRANT ALL PRIVILEGES ON DATABASE bank_reviews TO admin;

# CREATE SCHEMA bank_schema;


# -- Login using the new user:
# psql -U admin -d bank_reviews -h localhost


# -- Create the tables


# -- Banks table

# CREATE TABLE bank_schema.banks (
#     bank_id SERIAL PRIMARY KEY,
#     bank_name VARCHAR(255) UNIQUE,
#     app_name VARCHAR(255)
# );


# -- Reviews table

# CREATE TABLE bank_schema.reviews (
#     review_id SERIAL PRIMARY KEY,
#     bank_id INT REFERENCES bank_schema.banks(bank_id),
#     review_text TEXT,
#     rating INT,
#     review_date DATE,
#     sentiment_label VARCHAR(20),
#     sentiment_score FLOAT,
#     source VARCHAR(50)
# );



# --check the tables

# \d bank_schema.*


# -- List all tables

# \dt bank_schema.*

# -- Describe a table (check schema)

# \d bank_schema.banks


# -- Delete all records

# TRUNCATE TABLE bank_schema.reviews RESTART IDENTITY CASCADE;


In [40]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")



Current working directory: c:\Users\mihret.mekbebe\Documents\REQUESTS\KAIM\Customer-Experience-Analytics-for-Fintech-Apps-Week2\notebooks


In [41]:
import pandas as pd
import psycopg2

In [53]:
# Read CSV files into pandas DataFrames
df_banks = pd.read_csv("../data/processed/reviews_processed.csv")
df_reviews = pd.read_csv("../data/processed/reviews_with_sentiment.csv")
print(df_banks["bank_name"].value_counts())
df_reviews.head()

bank_name
Bank of Abyssinia              400
Commercial Bank of Ethiopia    400
Dashen Bank                    400
Name: count, dtype: int64


,review_id,review_text,rating,review_date,bank_code,bank_name,source,bert_sentiment,bert_score,bert_numeric
0,0,🙏👍,5,2025-11-29,BOA,Bank of Abyssinia,Google Play,NEGATIVE,0.697057,-1
1,1,Very Good,5,2025-11-28,BOA,Bank of Abyssinia,Google Play,POSITIVE,0.999852,1
2,2,goof,5,2025-11-28,BOA,Bank of Abyssinia,Google Play,NEGATIVE,0.997528,-1
3,3,good!,5,2025-11-28,BOA,Bank of Abyssinia,Google Play,POSITIVE,0.999827,1
4,4,good jop,5,2025-11-27,BOA,Bank of Abyssinia,Google Play,POSITIVE,0.999841,1


In [54]:
# Connect to PostgreSQL database

# Establish a connection to PostgreSQL using psycopg2
conn = psycopg2.connect(
    host="localhost",              # PostgreSQL is running on your machine
    database="bank_reviews",       # Database name 
    user="admin",                  # PostgreSQL username
    password="admin123"            # PostgreSQL password
)

# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

In [55]:
# -------------------------------------------------------
# Insert data into banks table
# -------------------------------------------------------
unique_banks = df_banks["bank_name"].drop_duplicates() # Get unique bank names

# Loop through each row in unique_banks and insert into banks table
for bank in unique_banks:

    # Execute SQL INSERT for each row
    # ON CONFLICT DO NOTHING avoids errors if the primary key already exists
    cur.execute(
        """
        INSERT INTO bank_schema.banks (bank_name)
        VALUES (%s)
        ON CONFLICT DO NOTHING;
        """,
        (bank,)
    )

conn.commit()   # Saves all INSERT operations permanently

print("Data inserted successfully!")

Data inserted successfully!


In [58]:
# renaming columns in df_reviews for clarity
df_reviews = df_reviews.rename(columns={
    "bert_sentiment": "sentiment_label",
    "bert_score": "sentiment_score"
})


In [59]:
# retrieve bank_id and bank_name from banks table to use for foreign key reference
bank_lookup = pd.read_sql("SELECT bank_id, bank_name FROM bank_schema.banks;", conn)
print(bank_lookup)

   bank_id                    bank_name
0        1            Bank of Abyssinia
1        2  Commercial Bank of Ethiopia
2        3                  Dashen Bank


C:\Users\mihret.mekbebe\AppData\Local\Temp\ipykernel_6400\1864249370.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bank_lookup = pd.read_sql("SELECT bank_id, bank_name FROM bank_schema.banks;", conn)


In [60]:
# merge df_merged with bank_lookup to get bank_id
df_merged = df_reviews.merge(bank_lookup, on="bank_name", how="left")

# Insert data into reviews table

# Loop through each row in df_merged
for _, row in df_merged.iterrows():

    # Insert each review into PostgreSQL
    cur.execute(
        """
        INSERT INTO bank_schema.reviews 
            (review_id, bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
        VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
        """,
        (
            int(row["review_id"]),       # Review ID (primary key)
            int(row["bank_id"]),   # Foreign key referencing banks table
            row["review_text"],          # Full text review
            int(row["rating"]),          # Numerical rating
            row["review_date"],          # Date string
            row["sentiment_label"],      # Sentiment label
            row["sentiment_score"],      # Sentiment score
            row["source"]                # Review source 
        )
    )

# -------------------------------------------------------
# Save changes and close connection
# -------------------------------------------------------




In [61]:
conn.commit()   # Saves all INSERT operations permanently
cur.close()     # Close the cursor
conn.close()    # Close the database connection